# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Graphlab Create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\e150003\AppData\Local\Temp\graphlab_server_1520176086.log.0


This non-commercial license of GraphLab Create for academic use is assigned to E150003@e.ntu.edu.sg and will expire on February 26, 2019.


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

In [4]:
sales.head()

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
7129300520,2014-10-13 00:00:00+00:00,221900.0,3.0,1.0,1180.0,5650,1.0,0
6414100192,2014-12-09 00:00:00+00:00,538000.0,3.0,2.25,2570.0,7242,2.0,0
5631500400,2015-02-25 00:00:00+00:00,180000.0,2.0,1.0,770.0,10000,1.0,0
2487200875,2014-12-09 00:00:00+00:00,604000.0,4.0,3.0,1960.0,5000,1.0,0
1954400510,2015-02-18 00:00:00+00:00,510000.0,3.0,2.0,1680.0,8080,1.0,0
7237550310,2014-05-12 00:00:00+00:00,1225000.0,4.0,4.5,5420.0,101930,1.0,0
1321400060,2014-06-27 00:00:00+00:00,257500.0,3.0,2.25,1715.0,6819,2.0,0
2008000270,2015-01-15 00:00:00+00:00,291850.0,3.0,1.5,1060.0,9711,1.0,0
2414600126,2015-04-15 00:00:00+00:00,229500.0,3.0,1.0,1780.0,7470,1.0,0
3793500160,2015-03-12 00:00:00+00:00,323000.0,3.0,2.5,1890.0,6560,2.0,0


* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [5]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [6]:
model_all = graphlab.linear_regression.create(
    sales,
    target='price',
    features=all_features,
    validation_set=None,
    verbose = False,
    l2_penalty=0.,
    l1_penalty=1e10)

Find what features had non-zero weight.

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

### 1. We learn weights on the entire house dataset, using an L1 penalty of 1e10 (or 5e2, if using scikit-learn). Some features are transformations of inputs; see the reading.

Which of the following features have been chosen by LASSO, i.e. which features were assigned nonzero weights? (Choose all that apply)

In [7]:
model_all['coefficients'][model_all['coefficients']['value'] != 0.0][:]

name,index,value,stderr
(intercept),None,274873.05595,None
bathrooms,None,8468.53108691,None
sqft_living,None,24.4207209824,None
sqft_living_sqrt,None,350.060553386,None
grade,None,842.068034898,None
sqft_above,None,20.0247224171,None


# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [8]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [9]:
import numpy as np

In [10]:
validation_rss = {}
for l1_penalty in np.logspace(1, 7, num=13):
    model = graphlab.linear_regression.create(
        training,
        target='price',
        features=all_features,
        validation_set=None,
        verbose = False,
        l2_penalty=0.,
        l1_penalty=l1_penalty)

    predictions = model.predict(validation)
    residuals = validation['price'] - predictions
    rss = (residuals*residuals).sum()

    validation_rss[l1_penalty] = rss

In [11]:
print min(validation_rss.items(), key=lambda x: x[1])

(10.0, 625766285142460.5)


### 2. We split the house sales dataset into training set, test set, and validation set and choose the l1_penalty that minimizes the error on the validation set.

In which of the following ranges does the best l1_penalty fall?

In [12]:
min(validation_rss.items(), key=lambda x: x[1])[0]

10.0

### 3. Using the best value of l1_penalty as mentioned in the previous question, how many nonzero weights do you have?

In [13]:
model = graphlab.linear_regression.create(
        training,
        target='price',
        features=all_features,
        validation_set=None,
        verbose = False,
        l2_penalty=0.,
        l1_penalty=10.0)

In [14]:
len(model['coefficients'][model['coefficients']['value'] != 0.0])

18

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [15]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [16]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [17]:
coef_dict = {}
for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(
        training,
        target ='price',
        features=all_features,
        validation_set=None,
        verbose=None,
        l2_penalty=0.,
        l1_penalty=l1_penalty)

    coef_dict[l1_penalty] = model['coefficients']['value'].nnz()

In [18]:
coef_dict

{100000000.0: 18,
 127427498.57031322: 18,
 162377673.91887242: 18,
 206913808.111479: 18,
 263665089.87303555: 17,
 335981828.6283788: 17,
 428133239.8719396: 17,
 545559478.1168514: 17,
 695192796.1775591: 17,
 885866790.4100832: 16,
 1128837891.6846883: 15,
 1438449888.2876658: 15,
 1832980710.8324375: 13,
 2335721469.0901213: 12,
 2976351441.631313: 10,
 3792690190.7322536: 6,
 4832930238.571753: 5,
 6158482110.6602545: 3,
 7847599703.514623: 1,
 10000000000.0: 1}

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [19]:
l1_penalty_min = -1e+99
for l1_penalty, non_zeros in coef_dict.items():
    if non_zeros <= max_nonzeros:
        continue
    
    l1_penalty_min = max(l1_penalty_min, l1_penalty)
    
l1_penalty_min

2976351441.631313

In [20]:
l1_penalty_max = 1e+99
for l1_penalty, non_zeros in coef_dict.items():
    if non_zeros >= max_nonzeros:
        continue
    
    l1_penalty_max = min(l1_penalty_max, l1_penalty)
    
l1_penalty_max

3792690190.7322536

### 4. We explore a wide range of l1_penalty values to find a narrow region of l1_penaty values where models are likely to have the desired number of non-zero weights (max_nonzeros=7).

What value did you find for l1_penalty_min?

If you are using GraphLab Create, enter your answer in simple decimals without commas (e.g. 1131000000), rounded to nearest millions.

If you are using scikit-learn, enter your answer in simple decimals without commas (e.g. 4313), rounded to nearest integer.

In [21]:
l1_penalty_min

2976351441.631313

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [22]:
l1_penalty_values = np.linspace(l1_penalty_min, l1_penalty_max, 20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [23]:
validation_rss = {}
for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(
        training, target='price',
        features=all_features,
        validation_set=None,
        verbose = False,
        l2_penalty=0.,
        l1_penalty=l1_penalty)

    predictions = model.predict(validation)
    residuals = validation['price'] - predictions
    rss = (residuals*residuals).sum()

    validation_rss[l1_penalty] = rss, model['coefficients']['value'].nnz()

In [24]:
validation_rss

{2976351441.631313: (966925692362085.2, 10),
 3019316638.952415: (974019450084554.6, 10),
 3062281836.273517: (981188367942453.8, 10),
 3105247033.5946193: (989328342459473.5, 10),
 3148212230.9157214: (998783211265890.1, 10),
 3191177428.2368236: (1008477167020094.4, 10),
 3234142625.5579257: (1018298780553819.4, 10),
 3277107822.879028: (1028247992205977.0, 10),
 3320073020.20013: (1034616909232827.4, 8),
 3363038217.521232: (1038554735941040.2, 8),
 3406003414.8423343: (1043237237871701.2, 8),
 3448968612.1634364: (1046937488751711.5, 7),
 3491933809.4845386: (1051147625612862.5, 7),
 3534899006.8056407: (1055992735342999.4, 7),
 3577864204.126743: (1060799531763286.9, 7),
 3620829401.447845: (1065707689498230.1, 6),
 3663794598.768947: (1069464335425585.2, 6),
 3706759796.0900493: (1073504549585599.1, 6),
 3749724993.4111514: (1077632775581415.2, 6),
 3792690190.7322536: (1081867592324112.2, 6)}

In [25]:
best_rss = 1e+99
for l1_penalty, (rss, non_zeros) in validation_rss.items():    
    if (non_zeros == max_nonzeros) and (l1_penalty < best_rss):
        best_rss = rss
        best_l1_penalty = l1_penalty
        
print best_rss, best_l1_penalty

1.05599273534e+15 3534899006.8056407


### 5. We then explore the narrow range of l1_penalty values between l1_penalty_min and l1_penalty_max.

What value of l1_penalty in our narrow range has the lowest RSS on the VALIDATION set and has sparsity equal to max_nonzeros?

If you are using GraphLab Create, enter your answer in simple decimals without commas (e.g. 1131000000), rounded to nearest millions.

If you are using scikit-learn, enter your answer in simple decimals without commas (e.g. 4342), rounded to nearest integer.

In [26]:
best_l1_penalty

3534899006.8056407

### 6. Consider the model learned with the l1_penalty found in the previous question. Which of the following features has non-zero coefficients? (Choose all that apply)

In [27]:
model = graphlab.linear_regression.create(
    training,
    target='price',
    features=all_features,
    validation_set=None,
    verbose = False,
    l2_penalty=0.,
    l1_penalty=best_l1_penalty)

In [28]:
print model["coefficients"][model["coefficients"]["value"] != 0.0][:]

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 226807.047234 |  None  |
|     bedrooms     |  None | 322.098888436 |  None  |
|    bathrooms     |  None | 15392.9565223 |  None  |
|   sqft_living    |  None | 31.9817554069 |  None  |
| sqft_living_sqrt |  None | 666.949904077 |  None  |
|      grade       |  None | 2747.52787565 |  None  |
|    sqft_above    |  None | 29.4061078857 |  None  |
+------------------+-------+---------------+--------+
[7 rows x 4 columns]

